In [3]:
import os

In [5]:
%pwd

'/Users/martonszentpali/mlops'

In [4]:
os.chdir("../")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [10]:
from MLops.constants import *
from MLops.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config   


In [15]:
import os
import urllib.request as request
import zipfile as zip
from MLops.utils.common import get_size
from MLops.utils import logger



In [16]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zip.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    
    

[2025-10-15 14:57:02,039: INFO: common: yaml file: configs/config.yaml loaded successfully]
[2025-10-15 14:57:02,042: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-15 14:57:02,043: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-15 14:57:02,043: INFO: common: created directory at: artifacts]
[2025-10-15 14:57:02,044: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-15 14:57:02,754: INFO: 962337281: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 25456
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "40512b9e96bd9e54b114a3cd6ad6b9bf562cb6e0f450e06afc5e073bdf486473"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3BA6:3E5436:173138:19BB90:68EF9A1E
Accept-Ranges: bytes
Date: Wed, 15